In [3]:
import sympy as sp

# define zero point
INF = (None, None)

def mod_inverse(x, p):
    """find inverse of mod p"""
    return sp.mod_inverse(x, p)

def point_addition(P, Q, a, b, p):
    """point addition on EC, return point, slope"""
    if P == INF or Q == INF:
        return (P if Q == INF else Q), None  # a + 0 = a

    if P == Q: 
        if P[1] == 0: 
            return INF, None
        # slope : (3x^2 + a) / (2y) mod p
        m = (3 * P[0]**2 + a) * mod_inverse(2 * P[1], p) % p
    else:  
        if P[0] == Q[0]: 
            if P[1] == Q[1]:  # P == Q
                return point_addition(P, P, a, b, p) 
            else: 
                return INF, None
        # slope : (y2 - y1) / (x2 - x1) mod p
        m = (Q[1] - P[1]) * mod_inverse(Q[0] - P[0], p) % p
    
    # result
    x_r = (m**2 - P[0] - Q[0]) % p
    y_r = (m * (P[0] - x_r) - P[1]) % p
    return (x_r, y_r), m

def ec_mult(k, P, a, p):
    """scalar multiplication"""
    result = INF
    addend = P
    while k > 0:
        if k % 2 == 1:
            result, _ = point_addition(result, addend, a, b, p)
        addend, _ = point_addition(addend, addend, a, b, p)
        k //= 2
    return result

def is_on_curve(point, a, b, p):
    """checking point is on EC"""
    if point == INF:
        return True
    x, y = point
    return (y**2 - (x**3 + a*x + b)) % p == 0

In [4]:
# Parameters
a, b, p = 1, -1, 23  # EC y^2 = x^3 + x - 1 (mod 23)
n = p-1
Q=(7,2)
P=(8,6)


# check
assert is_on_curve(P, a, b, p), "P is not on the curve"
assert is_on_curve(Q, a, b, p), "Q is not on the curve"

# point addition
R, _ = point_addition(P, Q, a, b, p)
print(f"P + Q = {R}")

# scalar multiplication: 3 * P
three_P = ec_mult(3, P, a, p)
print(f"3 * P = {three_P}")

P + Q = (mpz(1), mpz(22))
3 * P = (mpz(9), mpz(1))


In [5]:
# P = k * Q using brute force
def brute_force(P, Q, a, b, p):
    k = 1
    current = Q
    while True:
        if current == P:
            break
        
        current, _ = point_addition(current, Q, a, b, p) 
        k += 1
    return k
k = brute_force(P, Q, a, b, p)
print(f'brute_force method: P = {k} * Q')

brute_force method: P = 16 * Q


In [7]:
import math

def baby_step_giant_step(P, Q, a, b, p, n): #Baby-step giant-step
    """Baby-step Giant-step algorithm"""
    if P == INF or Q == INF:
        return (P if Q == INF else Q), None  
    
    m = math.ceil(math.sqrt(n))  # m = ceil(sqrt(n))
    
    # Baby-step: {0*Q, 1*Q, ..., (m-1)*Q} Table
    baby_steps = {}
    current = INF
    for i in range(m):
        if current != INF:
            baby_steps[current] = i
        current, _ = point_addition(current, Q, a, b, p)
    
    # Giant-step: P - i*m*Q
    mQ = ec_mult(m, Q, a, p)  # m*Q
    inv_mQ = (mQ[0], -mQ[1] % p) if mQ != INF else INF  # -m*Q
    current = P
    for j in range(m):
        if current in baby_steps:
            i = baby_steps[current]
            k = j * m + i
            if k < n:
                return k
        if current == INF:
            break
        current, _ = point_addition(current, inv_mQ, a, b, p)
    
    return None  # no solution

# example
k = baby_step_giant_step(P, Q, a, b, p, n)
if k is not None:
    print(f'Baby-step giant-step method: P = {k} * Q')
else:
    print('해가 없습니다.')


P_find = ec_mult(k, P, a, p)
print(f'P: {P}, P_find: {P_find}')

Baby-step giant-step method: P = 16 * Q
P: (8, 6), P_find: (mpz(8), mpz(6))
